In [1]:
import pandas as pd
import geopandas as gpd
import os
import pyrosm as pyrosm
import time

data = os.getenv('data')

/home/nicholasjones/.local/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.14.1). Conversions between both will be slow.
  warnings.warn(


#### 1. Download data in PBF format

In [2]:
pyrosm.data.available['regions']['europe'][:5]

['albania', 'andorra', 'austria', 'azores', 'belarus']

In [3]:
countries = ['albania',
 'bosnia_herzegovina',
 'bulgaria',
 'croatia']

In [4]:
def get_pth(country):
    return(data + '/osm/{}'.format(country))

In [5]:
def get_data(country):
    t0 = time.time()
    pth = get_pth(country)
    if not os.path.exists(pth):
        os.mkdir(pth)
    pyrosm.get_data(country,directory=pth)
    t1 = time.time()
    print("took {:.0f} seconds".format(t1-t0))

In [65]:
get_data('andorra')

Downloaded Protobuf data 'andorra-latest.osm.pbf' (1.67 MB) to:
'/home/nicholasjones/Data/osm/andorra/andorra-latest.osm.pbf'
andorra downloaded to /home/nicholasjones/Data/osm/andorra in 17 seconds


#### 2. Get schools and hospitals from it

In [6]:
def get_pois(country,pois,in_pth = False, convert_polygons_to_points = True, save_shp = True):
    t0 = time.time()
    if not in_pth:
        try:
            in_pth = get_pth(country + "/{}-latest.osm.pbf".format(country))
            print(in_pth)
        except:
            print(in_pth + ' did not work')
            print('check path to PBF file')
    osm = pyrosm.OSM(in_pth)
    
    if type(pois) != list:
        pois = [pois]
    filter = {'amenity': pois}
    gdf = osm.get_pois(custom_filter=filter)[['name','geometry','amenity']]
    
    if convert_polygons_to_points:
        gdf.geometry = [geom.centroid if geom.type in ['Polygon','MultiPolygon'] else geom for geom in gdf.geometry]
        
    if save_shp:
        out_pth = get_pth(country + '/{}_{}.shp'.format(country,'_'.join(pois)))
        gdf.to_file(out_pth)
        
        
    t1=time.time()
    print("retrieved {} POIs in {:,.0f} seconds".format(len(gdf), t1-t0))
    return(gdf)

In [13]:
#gdf = get_pois('andorra',['hospital','school'])
#gdf.head()

#### 3. Do it for some countries

In [21]:
gdf = get_pois('montenegro','hospital')

/home/nicholasjones/Data/osm/montenegro/montenegro-latest.osm.pbf
retrieved 62 POIs in 10 seconds


In [22]:
get_pois('albania','school')

/home/nicholasjones/Data/osm/albania/albania-latest.osm.pbf
retrieved 591 POIs in 16 seconds


,name,geometry,amenity
0,Arsakeio e Tiranes,POINT (19.85806 41.28925),school
1,Shkolla 9 Vjeçare - Bërxullë,POINT (19.69873 41.38418),school
2,Qazim Shehu,POINT (20.44830 42.05309),school
3,Shkolla e ciklit të ulët (Kodër-Bërxullë).,POINT (19.68800 41.38236),school
4,None,POINT (20.47636 40.97651),school
...,...,...,...
586,Shkolla 9-të vjeçare Himar,POINT (19.74570 40.10214),school
587,Gjimnazi i Himarës,POINT (19.74058 40.10170),school
588,Liceu Artistik “Jordan Misja”,POINT (19.82506 41.32150),school
589,"Shkolla 9-vjeçare ""Shaqe Mazreku""",POINT (19.44733 41.31786),school


In [23]:
get_pois('albania','hospital')

/home/nicholasjones/Data/osm/albania/albania-latest.osm.pbf
retrieved 99 POIs in 17 seconds


,name,geometry,amenity
0,None,POINT (20.41603 42.08176),hospital
1,None,POINT (19.56194 40.72205),hospital
2,"""Rrahim Xhika""",POINT (20.42672 41.68726),hospital
3,None,POINT (20.22813 40.50148),hospital
4,Qendra shendetesore Nr.7,POINT (19.79459 41.33858),hospital
...,...,...,...
94,Spitali Amerikan Durrës,POINT (19.44586 41.32577),hospital
95,Sanotoriumi Shefqet Ndroqi,POINT (19.84410 41.31395),hospital
96,Salus,POINT (19.76208 41.34435),hospital
97,None,POINT (19.74467 40.10202),hospital


In [ ]:
get_pois('croatia','school')

/home/nicholasjones/Data/osm/croatia/croatia-latest.osm.pbf


In [ ]:
get_pois('croatia','hospital')

/home/nicholasjones/Data/osm/croatia/croatia-latest.osm.pbf


In [ ]:
get_pois('serbia','school')

In [ ]:
get_pois('serbia','hospital')

In [ ]:
get_pois('macedonia','school')

In [ ]:
get_pois('macedonia','hospital')

In [5]:
get_pois('bosnia','school')

/home/nicholasjones/Data/osm/bosnia/bosnia-latest.osm.pbf
retrieved 819 POIs in 34 seconds


,name,geometry,amenity
0,OŠ Rade Kondić,POINT (16.84024 44.97496),school
1,Četvrta OŠ,POINT (18.81130 44.87614),school
2,Osnovna škola Mahala,POINT (16.66884 44.75616),school
3,"OŠ ""Simo Petrović""",POINT (18.59790 44.96760),school
4,"OŠ ""Petar Kočić""",POINT (17.30723 45.05632),school
...,...,...,...
814,None,POINT (18.47229 45.05852),school
815,"Osnovna škola ""Vitez"" Vitez",POINT (17.79008 44.15148),school
816,Osnovna škola Ivana Mažuranića,POINT (17.23370 43.71541),school
817,Osnovna škola Milan Rakić,POINT (17.19481 44.69902),school


In [6]:
get_pois('bosnia','hospital')

/home/nicholasjones/Data/osm/bosnia/bosnia-latest.osm.pbf
retrieved 57 POIs in 62 seconds


,name,geometry,amenity
0,Univerzitetska bolnica Foča,POINT (18.78229 43.52731),hospital
1,Opća bolnica dr. Mustafa Beganović Gračanica,POINT (18.30313 44.69846),hospital
2,Bolnica Zvornik,POINT (19.10137 44.38728),hospital
3,Centar za srce BH Tuzla,POINT (18.70669 44.52788),hospital
4,Altamedica,POINT (17.77860 44.16875),hospital
5,Diva - Ginekološka klinika,POINT (17.19241 44.75857),hospital
6,Zdravstvena stanica Bijela,POINT (18.56204 44.78840),hospital
7,Ambulanta,POINT (18.70757 44.75832),hospital
8,None,POINT (18.35905 44.38016),hospital
9,Ortofizik,POINT (18.81674 44.87403),hospital


In [ ]:
get_pois('greece','school')

/home/nicholasjones/Data/osm/greece/greece-latest.osm.pbf


In [ ]:
get_pois('greece','hospital')

/home/nicholasjones/Data/osm/greece/greece-latest.osm.pbf


In [ ]:
get_pois('slovakia','school')
get_pois('slovakia','hospital')

/home/nicholasjones/Data/osm/slovakia/slovakia-latest.osm.pbf


In [ ]:
get_pois('slovenia','school')
get_pois('slovenia','school')

In [ ]:
get_pois('austria','school')
get_pois('austria','hospital')
